In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [ ]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [ ]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [ ]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

In [ ]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

In [ ]:
df_an.columns

In [ ]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

In [ ]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

In [ ]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

In [ ]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [ ]:
df_an.columns

In [ ]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
    if df_an.SIGNAL[i] != NONE:
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [ ]:
print(len(X))
print(len(Y))
print(len(indeces))

In [ ]:
print(len(feature))

In [ ]:
SPLIT_INDEX = int(0.8*len(X))

In [ ]:

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [ ]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

In [ ]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [ ]:
# help(MLPClassifier)

In [ ]:
clf = MLPClassifier( solver='adam', alpha=1e-5,learning_rate_init=1e-4,
#                      tol=0.0000010,
                    hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
y_perdict = clf.predict(X_test)

In [ ]:
confusion_matrix(Y_test, y_perdict)

In [ ]:
pickle.dump(clf, open(f"{pair}.pickle", "wb"))

In [ ]:
(Y_test==y_perdict).sum()/len(y_perdict)

In [ ]:
print(len(indeces_test))
print(len(y_perdict))
# y_perdict

In [ ]:
df_an.columns

In [ ]:
indeces_test[0]

In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


In [ ]:
df_test

In [ ]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
# orginal 
# df_res_m5['balance'] = 100
# loss_margin = 0.03
# perv_balance = 100

# # for index, row in df_res_m5.iterrows():
# for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
#     if(df_res_m5.result[i] == -1):
#         df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
#     elif(df_res_m5.result[i] == 2):
#         profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
#     perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5

In [ ]:
df_res_m5.result.value_counts()

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

In [2]:
#  cross test
pair_cross = "EURUSD"
NB_H1_CANDLES_CROSS = 2000
rates = Rates(pair_cross, NB_H1_CANDLES_CROSS, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an_cross = rates.get_rates_from_now()
df_an_cross.drop(NB_H1_CANDLES_CROSS-1,inplace=True) 
df_an_cross.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an_cross = apply_patterns(df_an_cross)
for cp in candle_patterns:
    df_an_cross[cp] = df_an_cross[cp].astype(float)

df_an_cross = RSI(df_an_cross)
df_an_cross = BollingerBands(df_an_cross)
df_an_cross = ATR(df_an_cross)
df_an_cross = KeltnerChannels(df_an_cross)
df_an_cross = MACD(df_an_cross)

df_an_cross['pivots_l'] = False
df_an_cross['pivots_h'] = False

df_an_cross['low_perv'] = df_an_cross.low.shift(1)
df_an_cross['high_perv'] = df_an_cross.high.shift(1)
df_an_cross['low_next'] = df_an_cross.low.shift(-1)
df_an_cross['high_next'] = df_an_cross.high.shift(-1)
df_an_cross['pivots_l'] = (df_an_cross.low_perv > df_an_cross.low) & (df_an_cross.low_next > df_an_cross.low)
df_an_cross['pivots_h'] = (df_an_cross.high_perv < df_an_cross.high) & (df_an_cross.high_next < df_an_cross.high)
df_an_cross.dropna(inplace=True)
df_an_cross.reset_index(drop=True, inplace=True)

df_an_cross.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2022-11-25 16:00:00,1.03727,1.03886,1.03549,1.03814,1.03727,1.03814,52.818991,78.635015,25.816024,...,1.035196,-0.000679,-0.000348,-0.000331,True,False,1.03614,1.03835,1.03764,1.04070
1,2022-11-25 17:00:00,1.03815,1.04070,1.03764,1.04067,1.03815,1.04067,16.666667,99.019608,82.352941,...,1.034942,-0.000547,-0.000393,-0.000154,False,False,1.03549,1.03886,1.03917,1.04114
2,2022-11-25 18:00:00,1.04067,1.04114,1.03917,1.04014,1.04014,1.04067,49.238579,76.142132,26.903553,...,1.034869,-0.000477,-0.000411,-0.000066,False,True,1.03764,1.04070,1.03990,1.04094
3,2022-11-25 19:00:00,1.04013,1.04094,1.03990,1.04059,1.04013,1.04059,22.115385,66.346154,44.230769,...,1.034980,-0.000383,-0.000405,0.000022,False,False,1.03917,1.04114,1.03969,1.04061
4,2022-11-25 20:00:00,1.04060,1.04061,1.03969,1.04016,1.04016,1.04060,51.086957,98.913043,47.826087,...,1.035197,-0.000338,-0.000391,0.000053,True,False,1.03990,1.04094,1.03988,1.04150


In [3]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

4e-05

In [24]:
BUY = 1
SELL = -1
NONE = 0

df_an_cross['SIGNAL'] = 0
df_an_cross['TP'] = 0
df_an_cross['SL'] = 0

for i in range(1,len(df_an_cross)):
    if df_an_cross.pivots_l[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.low[j] <= df_an_cross.low[i-1] ):
                break
            else:
                if(df_an_cross.high[j] > 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = BUY
                    df_an_cross['SL'][i] = df_an_cross.low[i-1] #- SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i] #+ SPREAD
                    break
    if df_an_cross.pivots_h[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.high[j] >= df_an_cross.high[i-1] ):
                break
            else:
                if(df_an_cross.low[j] < 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = SELL
                    df_an_cross['SL'][i] = df_an_cross.high[i-1] #+ SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\3328372866.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\3328372866.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
df_an_cross.pivots_l = df_an_cross.pivots_l.astype(float)
df_an_cross.pivots_h = df_an_cross.pivots_h.astype(float)

In [36]:
X_cross = []
Y_cross = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces_cross = []
for i in range(4,len(df_an_cross)):
    if df_an_cross.pivots_h[i-1] or df_an_cross.pivots_l[i-1]:
        print("im here")
        feature = []
        feature.append(df_an_cross.pivots_h[i-1])
        feature.append(df_an_cross.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an_cross[fc][i])
            feature.append(df_an_cross[fc][i-1])
            feature.append(df_an_cross[fc][i-2])
            feature.append(df_an_cross[fc][i-3])
            feature.append(df_an_cross[fc][i-4])

        X_cross.append(feature)
        Y_cross.append(df_an_cross.SIGNAL[i])
        indeces_cross.append(i)

im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here
im here


In [38]:
len(feature)

202

In [39]:
clf = pickle.load(open("GBPUSD.pickle", "rb"))

In [40]:
y_perdict_cross = clf.predict(X_cross)

In [41]:
confusion_matrix(Y_cross, y_perdict_cross)

array([[  0,   0,   0],
       [412,   0, 402],
       [  0,   0,   0]], dtype=int64)

In [35]:
(Y_cross==y_perdict_cross).sum()/len(y_perdict_cross)

0.0

In [42]:
df_test = df_an_cross[['time', 'open', 'high', 'low', 'close','TP','SL']]#.iloc[indeces_cross[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_cross):
    df_test.SIGNAL.loc[idx] = y_perdict_cross[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\56276388.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\56276388.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
rates = Rates(pair_cross, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    25545 non-null  datetime64[ns]
 1   open    25545 non-null  float64       
 2   high    25545 non-null  float64       
 3   low     25545 non-null  float64       
 4   close   25545 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 998.0 KB


In [44]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [45]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.657997399219766
 2.0    506
-1.0    263
Name: result, dtype: int64


In [46]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\1186191000.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
df_res_m5_plot = df_res_m5.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

C:\Users\meysam\AppData\Local\Temp\ipykernel_1984\597831068.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

